# ADABOUND Summary
> Adaptive Gradient Methods With Dynamic Bound Of Learning Rate by Iangchen Luo, Yuanhao Xiong, Yan Liu and Xu Sun

- toc: true 
- badges: true
- comments: true
- categories: [jupyter]
- image: images/adabound.png

[Paper Link](https://arxiv.org/abs/1902.09843)
## What did the authors want to achieve ?   
In recent years, a lot of different first order methods (ADAGRAD, RMSProp, ADAM, AMSGRAD) have been introduced for training deep neural nets in order to combat the uniform gradient scaling of SGD and it's slower training speed. Experiments and the release of state of the art CV and NLP models - which use traditional SGD (often with momentum) -  prove that these optimizers generalize poorly in comparison to classic SGD. Sometimes they don't even converge, which is largely because of unstable and extreme learning rates.  
   
The authors therefore provide new, dynamically bounded versions of AMSGRAD and ADAM, called AMSBOUND and ADABOUND. These new techniques can be seen as adaptive methods during early training, which smoothly transform into SGD/SGD with momentum as training time increases. 



## Methods



### Implementation 
An implementation of ADABOUND and ADABOUNDW can be found [here](https://github.com/Luolc/AdaBound/blob/master/adabound/adabound.py). We can see that the implementation is very similar to the standard ADAM optimizer in the Pytorch source code. The difference is the if/else condition in the end which keeps track of the maximum 2nd moment running average, as well as the bounding at the end of the ```step``` function. ADABOUNDW also adds weight decay at the end. 


## Results

